In [1]:
import requests
from finnhub import client as Finnhub
from pandas.io.json import json_normalize
import json
import requests
import pandas as pd
import pygsheets
import sys
sys.path.append('.')

import investing_functions as f

gc = pygsheets.authorize(service_file=r"google_spreadsheets_api_json_credentials")

# Import static variables
symbols_array = ['AAL', 'VIAC', 'M', 'DKS', 'MAC', 'CARS']
gsheet_id = 'https://docs.google.com/spreadsheets/d/1fcH7_9m3MEIKcPj9JYjG0pYfOxXL-J9IaFygl_XjAO0/edit#gid=1901856452'

### Weekly Candels - several stocks

In [2]:
# Get weekly candles
df_all_stocks_weekly = f.get_stocks_data_weekly(symbols_array)

# Upload weekly candles df to Gsheet
tab_id = 'candels_data_weekly'
f.export_to_gsheets(gsheet_id, tab_id, df_all_stocks_weekly)

print('All Stocks Weekly Data is uploaded to GSheets')

All Stocks Weekly Data is uploaded to GSheets


## Daily Candels - several stocks

In [3]:
# Get daily candles
df_all_stocks_daily = f.get_stocks_data_daily(symbols_array)

# Upload daily candles df to Gsheet
tab_id = 'candels_data_daily'
f.export_to_gsheets(gsheet_id, tab_id, df_all_stocks_daily)

print('All Stocks Data is uploaded to GSheets')

All Stocks Data is uploaded to GSheets


## Recommendation - several stocks

In [4]:
# Get Recommendation Data
df_all_recommendations = f.get_recommendations_data(symbols_array)

# Upload Recommendation Data to Gsheet
tab_id = 'recommendation_data'
f.export_to_gsheets(gsheet_id, tab_id, df_all_recommendations)

print('All Stocks Data is uploaded to GSheets')

All Stocks Data is uploaded to GSheets


## News - several stocks

In [5]:
# Get Recommendation Data
df_all_news = f.get_company_news(symbols_array)

# Upload Recommendation Data to Gsheet
tab_id = 'news_data'
f.export_to_gsheets(gsheet_id, tab_id, df_all_news)

print('All News Data is uploaded to GSheets')

All News Data is uploaded to GSheets


## Dividends

In [6]:
date_start = '2010-01-01'
date_end = '2020-12-31'
r = requests.get('https://finnhub.io/api/v1/stock/dividend?symbol={0}&from={1}&to={2}&token={3}'.format(symbol, date_start, date_end, token))
p = r.json()
df_dividend = pd.read_json(json.dumps(p))
df_dividend['date'] = pd.to_datetime(df_dividend['date'], unit='s')
df_dividend['symbol'] = symbol

################################################
# Заливаем все в sheet
################################################

sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fcH7_9m3MEIKcPj9JYjG0pYfOxXL-J9IaFygl_XjAO0/edit#gid=1901856452')
wks = sh.worksheet_by_title("dividends_data")
wks.clear()
wks.set_dataframe(df_dividend,(1,1), nan='')

NameError: name 'symbol' is not defined

## Margin Metrics

In [ ]:
# Get Margin Metrics Data
df_margin_metrics_all = f.get_margin_metrics(symbols_array)

# Upload Margin Metrics to Gsheet
tab_id = 'margin_metrics'
f.export_to_gsheets(gsheet_id, tab_id, df_margin_metrics_all)

print('All Margin Metrics is uploaded to GSheets')

## Upgrade/Downgrade

In [ ]:
r = requests.get('https://finnhub.io/api/v1/stock/upgrade-downgrade?symbol={0}&token={1}'.format(symbol, token))
p = r.json()
df_upgrade = pd.read_json(json.dumps(p))
df_upgrade['gradeTime'] = pd.to_datetime(df_upgrade['gradeTime'], unit='s')
df_upgrade = df_upgrade.loc[:, ['symbol', 'company', 'gradeTime', 'action', 'fromGrade', 'toGrade']]


################################################
# Заливаем все в sheet
################################################

sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fcH7_9m3MEIKcPj9JYjG0pYfOxXL-J9IaFygl_XjAO0/edit#gid=1901856452')
wks = sh.worksheet_by_title("upgrade_downgrade_data")
wks.clear()
wks.set_dataframe(df_upgrade,(1,1), nan='')

## Company news

In [ ]:
symbol = 'AAL'
token = 'finnhub_token_here'
r = requests.get('https://finnhub.io/api/v1/news/{0}?token={1}'.format(symbol, token))
p = r.json()
df_news = pd.read_json(json.dumps(p))
# df_upgrade['gradeTime'] = pd.to_datetime(df_upgrade['gradeTime'], unit='s')
# df_upgrade = df_upgrade.loc[:, ['symbol', 'company', 'gradeTime', 'action', 'fromGrade', 'toGrade']]
df_news.head(5)

columns = ['category', 'datetime', 'headline', 'id', 'image', 'related', 'source', 'summary', 'url']

################################################
# Заливаем все в sheet
################################################

# sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fcH7_9m3MEIKcPj9JYjG0pYfOxXL-J9IaFygl_XjAO0/edit#gid=1901856452')
# wks = sh.worksheet_by_title("news_data")
# wks.clear()
# wks.set_dataframe(df_news,(1,1), nan='')

In [ ]:
r = requests.get('https://finnhub.io/api/v1/calendar/earnings?from=2010-01-01&to=2020-04-15&symbol={0}&token={1}'.format(symbol, token))
p = r.json()
df_earnings = pd.read_json(json.dumps(p))
df_earnings_calendar = json_normalize(df_earnings['earningsCalendar']) 
df_earnings_calendar